In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
import os

# JSONL 파일 로드 (lines=True 필수)
# 파일 경로를 실제 파일 경로로 바꿔주세요.
df = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출 (여기서 명시적으로 .copy()를 사용하여 새로운 DataFrame을 만듭니다)
df_processed = df[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# user_id와 business_id를 연속적인 정수 ID로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

# .loc를 사용하여 값을 할당하여 경고를 방지합니다.
df_processed.loc[:, 'user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed.loc[:, 'business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)

# 1. 먼저 학습+검증 세트와 테스트 세트로 8:2 비율로 분할
# (7+1):2 = 8:2
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)

# 2. 학습+검증 세트를 다시 학습 세트와 검증 세트로 7:1 비율로 분할
# val_size_ratio는 train_val_df의 1/8이므로, 전체 데이터의 1/10이 됩니다.
val_size_ratio = 1 / 8
train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=42)

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")

# --- PyTorch Dataset 및 DataLoader 정의 ---
class ReviewDataset(Dataset):
    def __init__(self, df):
        self.user_ids = torch.tensor(df['user_encoded'].values, dtype=torch.long)
        self.business_ids = torch.tensor(df['business_encoded'].values, dtype=torch.long)
        self.sentiment_vectors = torch.tensor(np.array(df['sentiment_vector'].tolist()), dtype=torch.float)
        self.stars = torch.tensor(df['stars'].values, dtype=torch.float)

    def __len__(self):
        return len(self.stars)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.business_ids[idx], self.sentiment_vectors[idx], self.stars[idx]

# --- 모델 아키텍처 정의 (이전과 동일) ---
class CustomerRestaurantInteractionModule(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim, mlp_dims):
        super(CustomerRestaurantInteractionModule, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        layers = []
        input_dim = embedding_dim * 2
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = mlp_dims[-1] if mlp_dims else embedding_dim * 2

    def forward(self, user_ids, business_ids):
        user_vec = self.user_embedding(user_ids)
        business_vec = self.business_embedding(business_ids)
        combined_vec = torch.cat((user_vec, business_vec), dim=1)
        interaction_features = self.mlp(combined_vec)
        return interaction_features

class ReviewAspectModule(nn.Module):
    def __init__(self, sentiment_vector_dim, mlp_dims):
        super(ReviewAspectModule, self).__init__()
        layers = []
        input_dim = sentiment_vector_dim
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = mlp_dims[-1] if mlp_dims else sentiment_vector_dim

    def forward(self, sentiment_vectors):
        aspect_features = self.mlp(sentiment_vectors)
        return aspect_features

class AATRec(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim,
                 user_biz_mlp_dims, aspect_mlp_dims, final_mlp_dims,
                 sentiment_vector_dim=15):
        super(AATRec, self).__init__()
        self.customer_restaurant_interaction_module = CustomerRestaurantInteractionModule(
            num_users, num_businesses, embedding_dim, user_biz_mlp_dims
        )
        self.review_aspect_module = ReviewAspectModule(
            sentiment_vector_dim, aspect_mlp_dims
        )
        final_input_dim = self.customer_restaurant_interaction_module.output_dim + \
                          self.review_aspect_module.output_dim
        layers = []
        input_dim = final_input_dim
        for dim in final_mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        layers.append(nn.Linear(input_dim, 1))
        self.prediction_mlp = nn.Sequential(*layers)

    def forward(self, user_ids, business_ids, sentiment_vectors):
        user_biz_features = self.customer_restaurant_interaction_module(user_ids, business_ids)
        aspect_features = self.review_aspect_module(sentiment_vectors)
        combined_features = torch.cat((user_biz_features, aspect_features), dim=1)
        predicted_rating = self.prediction_mlp(combined_features)
        # FIX: Ensure the output is always a 1D tensor, not a 0D scalar
        return predicted_rating.view(-1) # Use view(-1) to flatten to 1D tensor

# --- 하이퍼파라미터 설정 ---
embedding_dim = 64
user_biz_mlp_dims = [128, 64]
aspect_mlp_dims = [64, 32]
final_mlp_dims = [64, 32]
learning_rate = 0.001
epochs = 50 # 조기 종료를 고려하여 충분히 큰 에폭 설정
batch_size = 256

# --- 조기 종료(Early Stopping) 설정 ---
patience = 5 # 검증 RMSE가 개선되지 않아도 기다릴 에폭 수
min_delta = 0.001 # 검증 RMSE가 개선되었다고 판단할 최소 변화량
best_val_rmse = float('inf') # 초기 최적 검증 RMSE (무한대로 설정)
epochs_no_improve = 0 # 검증 RMSE가 개선되지 않은 에폭 수 카운터
model_path = 'best_aatrec_model.pt' # 최적 모델 저장 경로

# 데이터셋 및 DataLoader 생성
train_dataset = ReviewDataset(train_df)
val_dataset = ReviewDataset(val_df)
test_dataset = ReviewDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 인스턴스 생성
model = AATRec(num_users, num_businesses, embedding_dim,
               user_biz_mlp_dims, aspect_mlp_dims, final_mlp_dims,
               sentiment_vector_dim=15)

# 손실 함수 및 옵티마이저
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# --- 학습 루프 (조기 종료 추가) ---
print("모델 학습 시작...")
for epoch in range(epochs):
    # --- 학습 단계 ---
    model.train()
    total_train_loss = 0
    for user_ids, business_ids, sentiment_vectors, stars in train_loader:
        optimizer.zero_grad()
        predictions = model(user_ids, business_ids, sentiment_vectors)
        loss = criterion(predictions, stars)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # --- 검증 단계 ---
    model.eval()
    total_val_loss = 0
    val_predictions = []
    val_true_ratings = []
    with torch.no_grad():
        for user_ids, business_ids, sentiment_vectors, stars in val_loader:
            predictions = model(user_ids, business_ids, sentiment_vectors)
            loss = criterion(predictions, stars)
            total_val_loss += loss.item()
            val_predictions.extend(predictions.tolist())
            val_true_ratings.extend(stars.tolist())

    current_val_rmse = np.sqrt(mean_squared_error(val_true_ratings, val_predictions))

    print(f"Epoch {epoch+1}/{epochs}, "
          f"Train Loss: {total_train_loss / len(train_loader):.4f}, "
          f"Val Loss: {total_val_loss / len(val_loader):.4f}, "
          f"Val RMSE: {current_val_rmse:.4f}")

    # --- 조기 종료 로직 ---
    if current_val_rmse < best_val_rmse - min_delta: # 검증 RMSE가 개선되었다면
        best_val_rmse = current_val_rmse
        epochs_no_improve = 0 # 개선이 있었으므로 카운트 초기화
        torch.save(model.state_dict(), model_path) # 최적 모델 가중치 저장
        print(f"  --> 검증 RMSE 개선됨. 최적 모델 저장됨: {best_val_rmse:.4f}")
    else: # 개선되지 않았다면
        epochs_no_improve += 1 # 카운트 증가
        print(f"  --> 검증 RMSE 개선되지 않음. 대기 중... ({epochs_no_improve}/{patience})")
        if epochs_no_improve == patience:
            print(f"조기 종료! {patience} 에폭 동안 검증 RMSE 개선이 없었습니다.")
            break # 학습 루프 종료

print("모델 학습 완료.")

# --- 최종 테스트 및 RMSE 계산 ---
# 최적의 모델 가중치를 로드하여 테스트
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print(f"최적의 모델 가중치 '{model_path}' 로드 완료.")
else:
    print(f"최적의 모델 가중치 '{model_path}'를 찾을 수 없습니다. 현재 모델 상태로 테스트를 진행합니다.")

model.eval()
test_predictions = []
true_ratings = []

with torch.no_grad():
    for user_ids, business_ids, sentiment_vectors, stars in test_loader:
        predictions = model(user_ids, business_ids, sentiment_vectors)
        test_predictions.extend(predictions.tolist()) # This should now work correctly
        true_ratings.extend(stars.tolist())

rmse = np.sqrt(mean_squared_error(true_ratings, test_predictions))
print(f"\n최종 테스트 RMSE (최적 모델 기준): {rmse:.4f}")

# --- Precision@5 계산을 위한 추가 전처리 ---
# 1. 각 business_encoded별 평균 sentiment_vector 계산 (ABSA가 없는 아이템에 대한 대안)
#    주의: 실제 운영에서는 unseen item에 대한 sentiment_vector를 추론하거나, 해당 item의 평균 리뷰에서 추출해야 함
business_sentiment_map = train_df.groupby('business_encoded')['sentiment_vector'].apply(lambda x: np.mean(x.tolist(), axis=0)).to_dict()

# 모든 business_encoded의 리스트
all_business_encoded = np.arange(num_businesses)
# 기본 sentiment_vector (예: 모든 요소가 0 또는 전체 평균)
# Assuming sentiment_vector_dim = 15
default_sentiment_vector = np.zeros(15)

# 2. train_df에서 각 사용자별로 상호작용한 business_encoded ID 집합 생성
user_interacted_businesses = train_df.groupby('user_encoded')['business_encoded'].apply(set).to_dict()

# --- Precision@5 계산 함수 ---
def calculate_precision_at_k(model, test_df, num_users, all_business_encoded,
                             user_interacted_businesses, business_sentiment_map,
                             k=5, relevance_threshold=4.0, batch_size=256):
    model.eval()
    total_precision = 0
    num_users_with_recommendations = 0

    unique_test_users = test_df['user_encoded'].unique()

    with torch.no_grad():
        for user_encoded in unique_test_users:
            # 해당 사용자가 훈련 데이터에서 상호작용한 비즈니스 목록
            interacted_businesses = user_interacted_businesses.get(user_encoded, set())

            # 추천 후보 비즈니스 (훈련 데이터에 없는 모든 비즈니스)
            candidate_businesses = [
                b_id for b_id in all_business_encoded
                if b_id not in interacted_businesses
            ]

            if not candidate_businesses:
                # 이 사용자가 모든 비즈니스와 이미 상호작용했거나 후보가 없는 경우 스킵
                continue

            # 예측을 위한 데이터 준비
            user_ids_tensor = torch.tensor([user_encoded] * len(candidate_businesses), dtype=torch.long)
            business_ids_tensor = torch.tensor(candidate_businesses, dtype=torch.long)

            # 후보 비즈니스에 대한 sentiment_vector 준비
            candidate_sentiment_vectors = []
            for b_id in candidate_businesses:
                sentiment_vec = business_sentiment_map.get(b_id, default_sentiment_vector)
                candidate_sentiment_vectors.append(sentiment_vec)
            sentiment_vectors_tensor = torch.tensor(np.array(candidate_sentiment_vectors), dtype=torch.float)

            # 배치 단위로 예측 수행 (메모리 효율성)
            predictions_list = []
            for i in range(0, len(user_ids_tensor), batch_size):
                batch_user_ids = user_ids_tensor[i:i + batch_size]
                batch_business_ids = business_ids_tensor[i:i + batch_size]
                batch_sentiment_vectors = sentiment_vectors_tensor[i:i + batch_size]

                batch_preds = model(batch_user_ids, batch_business_ids, batch_sentiment_vectors)
                # FIX: batch_preds.tolist() will now correctly return a list because view(-1) ensures it's 1D
                predictions_list.extend(batch_preds.tolist())

            # 예측 결과를 비즈니스 ID와 연결
            predictions_dict = {b_id: pred for b_id, pred in zip(candidate_businesses, predictions_list)}

            # 상위 K개 추천 아이템 선정
            top_k_recommendations_encoded = sorted(predictions_dict.items(), key=lambda item: item[1], reverse=True)[:k]
            top_k_recommended_b_ids = [item[0] for item in top_k_recommendations_encoded]

            # 실제 관련성 확인 (테스트 세트 내에서만 확인)
            # test_df에서 해당 user_encoded가 실제로 높은 평점을 준 business_encoded를 찾습니다.
            user_actual_relevant_businesses = set(
                test_df[(test_df['user_encoded'] == user_encoded) & (test_df['stars'] >= relevance_threshold)]['business_encoded']
            )

            hits = 0
            for recommended_b_id in top_k_recommended_b_ids:
                if recommended_b_id in user_actual_relevant_businesses:
                    hits += 1

            # Precision 계산
            if k > 0:
                total_precision += (hits / k)
                num_users_with_recommendations += 1

    if num_users_with_recommendations == 0:
        return 0.0 # 추천할 사용자나 유효한 예측이 없는 경우
    return total_precision / num_users_with_recommendations

# --- Precision@5 계산 및 출력 ---
print("\nPrecision@5 계산 시작...")
precision_at_5 = calculate_precision_at_k(
    model, test_df, num_users, all_business_encoded,
    user_interacted_businesses, business_sentiment_map,
    k=5, relevance_threshold=4.0, batch_size=batch_size
)
print(f"Precision@5 (relevance_threshold=4.0): {precision_at_5:.4f}")

전체 데이터 수: 447796
학습 데이터 수: 313456 (70.00%)
검증 데이터 수: 44780 (10.00%)
테스트 데이터 수: 89560 (20.00%)
모델 학습 시작...
Epoch 1/50, Train Loss: 0.7677, Val Loss: 0.5071, Val RMSE: 0.7121
  --> 검증 RMSE 개선됨. 최적 모델 저장됨: 0.7121
Epoch 2/50, Train Loss: 0.4822, Val Loss: 0.4881, Val RMSE: 0.6986
  --> 검증 RMSE 개선됨. 최적 모델 저장됨: 0.6986
Epoch 3/50, Train Loss: 0.4493, Val Loss: 0.4776, Val RMSE: 0.6911
  --> 검증 RMSE 개선됨. 최적 모델 저장됨: 0.6911
Epoch 4/50, Train Loss: 0.4217, Val Loss: 0.4860, Val RMSE: 0.6972
  --> 검증 RMSE 개선되지 않음. 대기 중... (1/5)
Epoch 5/50, Train Loss: 0.3975, Val Loss: 0.4928, Val RMSE: 0.7020
  --> 검증 RMSE 개선되지 않음. 대기 중... (2/5)
Epoch 6/50, Train Loss: 0.3763, Val Loss: 0.4863, Val RMSE: 0.6974
  --> 검증 RMSE 개선되지 않음. 대기 중... (3/5)
Epoch 7/50, Train Loss: 0.3555, Val Loss: 0.5132, Val RMSE: 0.7164
  --> 검증 RMSE 개선되지 않음. 대기 중... (4/5)
Epoch 8/50, Train Loss: 0.3345, Val Loss: 0.5020, Val RMSE: 0.7085
  --> 검증 RMSE 개선되지 않음. 대기 중... (5/5)
조기 종료! 5 에폭 동안 검증 RMSE 개선이 없었습니다.
모델 학습 완료.
최적의 모델 가중치 'best_a